<a href="https://colab.research.google.com/github/inderpreetsingh01/ml_machine_coding/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# classification
import numpy as np
from collections import Counter

class KNNClassifier:
    def __init__(self, k=3, distance='euclidean'):
        self.k = k
        self.distance = distance

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _compute_distance(self, x1, x2):
        if self.distance == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.distance == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unsupported distance metric")

    def _predict_one(self, x):
        distances = [self._compute_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

    def predict(self, X):
        return np.array([self._predict_one(x) for x in X])

    def score(self, X, y):
        preds = self.predict(X)
        return np.mean(preds == y)

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features (very important for distance-based models!)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fit KNN
knn = KNNClassifier(k=5)
knn.fit(X_train, y_train)

# Evaluate
acc = knn.score(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

Test Accuracy: 1.0000


In [ ]:
# weighted KNN
class KNNClassifier:
    def __init__(self, k=3, distance='euclidean', weighted=False):
        self.k = k
        self.distance = distance
        self.weighted = weighted

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _compute_distance(self, x1, x2):
        if self.distance == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.distance == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unsupported distance metric")

    def _predict_one(self, x):
        distances = np.array([self._compute_distance(x, x_train) for x_train in self.X_train])
        k_indices = np.argsort(distances)[:self.k]
        k_labels = [self.y_train[i] for i in k_indices]

        if not self.weighted:
            # Unweighted majority vote
            return Counter(k_labels).most_common(1)[0][0]
        else:
            # Weighted vote (closer = more weight)
            weights = 1 / (distances[k_indices] + 1e-5)  # avoid div by zero
            label_weights = {}
            for label, weight in zip(k_labels, weights):
                label_weights[label] = label_weights.get(label, 0) + weight
            return max(label_weights.items(), key=lambda x: x[1])[0]

    def predict(self, X):
        return np.array([self._predict_one(x) for x in X])

    def score(self, X, y):
        preds = self.predict(X)
        return np.mean(preds == y)

In [ ]:
# KNN Regressor
class KNNRegressor:
    def __init__(self, k=3, distance='euclidean', weighted=False):
        self.k = k
        self.distance = distance
        self.weighted = weighted

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _compute_distance(self, x1, x2):
        if self.distance == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.distance == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unsupported distance metric")

    def _predict_one(self, x):
        distances = np.array([self._compute_distance(x, x_train) for x_train in self.X_train])
        k_indices = np.argsort(distances)[:self.k]
        k_values = [self.y_train[i] for i in k_indices]

        if not self.weighted:
            return np.mean(k_values)
        else:
            weights = 1 / (distances[k_indices] + 1e-5)
            return np.sum(weights * k_values) / np.sum(weights)

    def predict(self, X):
        return np.array([self._predict_one(x) for x in X])

    def score(self, X, y):
        preds = self.predict(X)
        return 1 - np.sum((preds - y) ** 2) / np.sum((y - np.mean(y)) ** 2)  # R² score

In [ ]:
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# For classification
X_cls, y_cls = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

knn_cls = KNNClassifier(k=5, weighted=True)
knn_cls.fit(X_train, y_train)
print("Weighted KNN Classifier Accuracy:", knn_cls.score(X_test, y_test))

# For regression (use any regression dataset like California housing, or Boston if available)
X_reg, y_reg = load_boston(return_X_y=True)  # use fetch_california_housing in newer sklearn
X_train, X_test, y_train, y_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

knn_reg = KNNRegressor(k=5, weighted=True)
knn_reg.fit(X_train, y_train)
print("Weighted KNN Regressor R² Score:", knn_reg.score(X_test, y_test))

In [ ]:
# cosine similarity

class KNNClassifier:
    def __init__(self, k=3, distance='euclidean', weighted=False):
        self.k = k
        self.distance = distance
        self.weighted = weighted

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _compute_distance(self, x1, x2):
        if self.distance == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.distance == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        elif self.distance == 'cosine':
            norm1 = np.linalg.norm(x1)
            norm2 = np.linalg.norm(x2)
            if norm1 == 0 or norm2 == 0:
                return 1.0  # Max distance if zero vector
            cosine_similarity = np.dot(x1, x2) / (norm1 * norm2)
            return 1 - cosine_similarity
        else:
            raise ValueError("Unsupported distance metric")

    def _predict_one(self, x):
        distances = np.array([self._compute_distance(x, x_train) for x_train in self.X_train])
        k_indices = np.argsort(distances)[:self.k]
        k_labels = [self.y_train[i] for i in k_indices]

        if not self.weighted:
            return Counter(k_labels).most_common(1)[0][0]
        else:
            weights = 1 / (distances[k_indices] + 1e-5)
            label_weights = {}
            for label, weight in zip(k_labels, weights):
                label_weights[label] = label_weights.get(label, 0) + weight
            return max(label_weights.items(), key=lambda x: x[1])[0]

    def predict(self, X):
        return np.array([self._predict_one(x) for x in X])

    def score(self, X, y):
        preds = self.predict(X)
        return np.mean(preds == y)

In [3]:
class KDNode:
    def __init__(self, point, axis, left=None, right=None):
        self.point = point       # A list/tuple like [x, y]
        self.axis = axis         # Splitting axis (0 for x, 1 for y, etc.)
        self.left = left         # KDNode
        self.right = right       # KDNode

In [4]:
import numpy as np

class KDTree:
    def __init__(self, points):
        self.k = len(points[0]) if points else 0  # dimensionality
        self.root = self._build(points)

    def _build(self, points, depth=0):
        if not points:
            return None

        axis = depth % self.k
        points.sort(key=lambda x: x[axis])
        median = len(points) // 2

        return KDNode(
            point=points[median],
            axis=axis,
            left=self._build(points[:median], depth + 1),
            right=self._build(points[median + 1:], depth + 1)
        )

    def nearest_neighbor(self, target):
        best = [None, float('inf')]  # [best_point, best_distance]

        def _search(node):
            if node is None:
                return

            point = node.point
            axis = node.axis
            dist = np.linalg.norm(np.array(point) - np.array(target))

            # Update best match if needed
            if dist < best[1]:
                best[0], best[1] = point, dist

            # Choose direction to search
            diff = target[axis] - point[axis]
            close, away = (node.left, node.right) if diff < 0 else (node.right, node.left)

            _search(close)

            # Check if we need to search the other side
            if abs(diff) < best[1]:
                _search(away)

        _search(self.root)
        return best[0], best[1]

In [ ]:
if __name__ == "__main__":
    points = [
        [2, 3],
        [5, 4],
        [9, 6],
        [4, 7],
        [8, 1],
        [7, 2]
    ]
    target = [9, 2]

    tree = KDTree(points)
    nearest_point, distance = tree.nearest_neighbor(target)
    print(f"Nearest to {target}: {nearest_point} (distance = {distance:.3f})")

In [ ]:
# with insert operation as well
class KDNode:
    def __init__(self, point, axis, left=None, right=None):
        self.point = point
        self.axis = axis
        self.left = left
        self.right = right

class KDTree:
    def __init__(self, points=None):
        self.k = len(points[0]) if points else 2  # default to 2D
        self.root = self._build(points) if points else None

    def _build(self, points, depth=0):
        if not points:
            return None

        axis = depth % self.k
        points.sort(key=lambda x: x[axis])
        median = len(points) // 2

        return KDNode(
            point=points[median],
            axis=axis,
            left=self._build(points[:median], depth + 1),
            right=self._build(points[median + 1:], depth + 1)
        )

    def insert(self, point):
        def _insert(node, point, depth):
            if node is None:
                return KDNode(point, depth % self.k)

            axis = node.axis
            if point[axis] < node.point[axis]:
                node.left = _insert(node.left, point, depth + 1)
            else:
                node.right = _insert(node.right, point, depth + 1)

            return node

        if self.root is None:
            self.root = KDNode(point, axis=0)
        else:
            _insert(self.root, point, 0)

    def nearest_neighbor(self, target):
        best = [None, float('inf')]

        def _search(node):
            if node is None:
                return

            point = node.point
            axis = node.axis
            dist = np.linalg.norm(np.array(point) - np.array(target))

            if dist < best[1]:
                best[0], best[1] = point, dist

            diff = target[axis] - point[axis]
            close, away = (node.left, node.right) if diff < 0 else (node.right, node.left)

            _search(close)
            if abs(diff) < best[1]:
                _search(away)

        _search(self.root)
        return best[0], best[1]


In [ ]:
if __name__ == "__main__":
    tree = KDTree([[3, 6], [17, 15], [13, 15], [6, 12], [9, 1], [2, 7], [10, 19]])

    print("Before insert:")
    nearest = tree.nearest_neighbor([9, 2])
    print(f"Nearest to [9, 2]: {nearest}")

    tree.insert([8, 3])
    print("\nAfter insert [8, 3]:")
    nearest = tree.nearest_neighbor([9, 2])
    print(f"Nearest to [9, 2]: {nearest}")

In [ ]:
# Notes on Insertions
# ✅ Works recursively like BST insert, with axis cycling.
# ❌ Does not rebalance the tree — over time, the tree may degrade in performance.
# To fix this, you’d need rebuilding periodically or use balanced KD-Tree variants.